# Import packages

In [1]:
import pandas as pd
from teradataml import *
import ast
import json
import torch
from teradataml.dataframe.copy_to import copy_to_sql
from transformers.generation import GenerationMixin

from modules.db import TeradataDatabase
from modules.models import( 
    OpusTranslationModelConfig, 
                        OpusTranslationModel, 
                        SentenceEmbeddingConfig, 
                        SentenceEmbeddingModel, 
)
from utils import clean_text, load_embedding_model, unicode_clean, load_translation_model
from constants import (
    CLEANED_TEST_DATA_PATH, TRAIN_VAL_DATA_PATH, CLASS_EMBEDDINGS_PATH, PRODUCT_TEST_EMBEDDINGS_PATH, 
    CLEANED_GPC_PATH, CLEANED_TEST_DATA_PATH, TEST_DATA_PATH, E5_LARGE_INSTRUCT_CONFIG_PATH, 
    OPUS_TRANSLATION_CONFIG_PATH, DATA_PATH, PRODUCT_TRAIN_EMBEDDINGS_PATH, VALIDATION_DATA_PATH,
    FULL_DATA_SET_DATA_PATH, PRODUCT_FULL_DATASET_EMBEDDINGS_PATH,  CLEANED_FULL_DATASET_DATA_PATH,
    CLASS_EMBEDDINGS_PATH_QWEN, PRODUCT_FULL_DATASET_EMBEDDINGS__QWEN_PATH,
    MWPD_DATA_SET_TRAIN, MWPD_DATA_SET_VAL, MWPD_DATA_SET_TEST, MWPD_FULL_DATASET
)


## Connect to database

In [3]:
td_db = TeradataDatabase()
td_db.connect()

In [ ]:
tdf = td_db.execute_query("Select * from demo_user.gpc_orig")
df = pd.DataFrame(tdf)


In [ ]:
df.head()

### Remvoing un-neccesary columns

In [ ]:
query = """
ALTER TABLE demo_user.gpc_orig 
DROP SegmentCode,
DROP FamilyCode,
DROP ClassCode,
Drop BrickCode,
DROP AttributeCode,
DROP AttributeTitle,
DROP AttributeDefinition,
DROP AttributeValueCode,
DROP AttributeValueTitle,
DROP AttributeValueDefinition;
"""

In [11]:
tdf = td_db.execute_query(query)

### Cleaning the GPC Columns

In [12]:
cleaning_query = """
UPDATE demo_user.gpc_orig
SET SegmentTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(SegmentTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    SegmentDefinition = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(SegmentDefinition, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    FamilyTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(FamilyTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    FamilyDefinition = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(FamilyDefinition, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    ClassTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(ClassTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    ClassDefinition = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(ClassDefinition, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    BrickTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(BrickTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    BrickDefinition_Includes = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(BrickDefinition_Includes, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    BrickDefinition_Excludes = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(BrickDefinition_Excludes, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' ')));

"""

In [13]:
tdf = td_db.execute_query(cleaning_query)

## Disconnect

In [ ]:
td_db.disconnect()

In [3]:
train_df = pd.read_json(MWPD_DATA_SET_TRAIN, lines=True)
val_df   = pd.read_json(MWPD_DATA_SET_VAL, lines=True)
test_df  = pd.read_json(MWPD_DATA_SET_TEST, lines=True)

full_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

full_df = full_df.drop_duplicates()

In [4]:
full_df.head()

,ID,Name,Description,CategoryText,URL,lvl1,lvl2,lvl3
0,549,Sterling Silver Angel Charm,This little angel charm is just heavenly,All Products,http://www.thecharmworks.com/product/CW-UA/Ste...,64000000_Personal Accessories,64010000_Personal Accessories,64010100_Jewellery
1,5664,HP Pavilion 23xi 58.40 cm (23�) IPS Monitor,"Share photos, videos, and games with everyone ...",Product,http://store.hp.com/UKStore/Merch/Product.aspx...,65000000_Computing,65010000_Computers/Video Games,65010700_Computer/Video Game Peripherals
2,3307,East Carolina Pirates Ladies Personalized Bask...,Feel like a bona fide member of East Carolina ...,East Carolina Pirates > East Carolina Pirates ...,http://eastcarolina.teamfanshop.com/COLLEGE_Ea...,67000000_Clothing,67010000_Clothing,67010800_Upper Body Wear/Tops
3,4609,"Tekonsha 90195 P3 Electric Brake Control, 1-4 ...",Receive free shipping on this item. Enter coup...,Vehicles & Parts > Vehicle Parts & Accessories,http://www.anythingtruck.com/product/755-90195...,77000000_Automotive,77010000_Automotive Accessories and Maintenance,77011200_Automotive Maintenance/Repair
4,7822,RN-XV WiFly Module - Wire Antenna,Description: The RN-XV module by Roving Networ...,Home :: Wireless ...,http://www.karlssonrobotics.com/cart/rn-xv-wif...,78000000_Electrical Supplies,78050000_Electronic Communication Components,78050100_Electronic Communication Components


In [6]:
full_df.drop(['ID', 'CategoryText', 'URL'], axis=1, inplace = True)

In [8]:
full_df.rename(columns={'lvl1': 'SegmentTitle', 'lvl2' : 'FamilyTitle', 'lvl3' : 'ClassTitle'}, inplace=True)

In [24]:
full_df.head()

,Name,Description,SegmentTitle,FamilyTitle,ClassTitle
0,Sterling Silver Angel Charm,This little angel charm is just heavenly,64000000_Personal Accessories,64010000_Personal Accessories,64010100_Jewellery
1,HP Pavilion 23xi 58.40 cm (23�) IPS Monitor,"Share photos, videos, and games with everyone ...",65000000_Computing,65010000_Computers/Video Games,65010700_Computer/Video Game Peripherals
2,East Carolina Pirates Ladies Personalized Bask...,Feel like a bona fide member of East Carolina ...,67000000_Clothing,67010000_Clothing,67010800_Upper Body Wear/Tops
3,"Tekonsha 90195 P3 Electric Brake Control, 1-4 ...",Receive free shipping on this item. Enter coup...,77000000_Automotive,77010000_Automotive Accessories and Maintenance,77011200_Automotive Maintenance/Repair
4,RN-XV WiFly Module - Wire Antenna,Description: The RN-XV module by Roving Networ...,78000000_Electrical Supplies,78050000_Electronic Communication Components,78050100_Electronic Communication Components


In [10]:
full_df.to_csv(MWPD_FULL_DATASET, index=False, encoding="utf-8")

In [12]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16119 entries, 0 to 16118
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          16119 non-null  object
 1   Description   16119 non-null  object
 2   SegmentTitle  16119 non-null  object
 3   FamilyTitle   16119 non-null  object
 4   ClassTitle    16119 non-null  object
dtypes: object(5)
memory usage: 629.8+ KB


In [23]:
df = full_df[full_df["SegmentTitle"].str.contains("Food/Beverage", case=False, na=False)]
count_food = len(df)
print((count_food + 250) / 16119 )    

0.03902227185309262


In [ ]:
df.head(10) 

,Name,Description,SegmentTitle,FamilyTitle,ClassTitle
78,Legacy 44 Servings Freeze-Dried Pineapple Pouc...,Add a whole new element to your bulk emergency...,50000000_Food/Beverage/Tobacco,50100000_Fruits/Vegetables/Nuts/Seeds Prepared...,50102100_Vegetables � Prepared/Processed
83,NFL Football Kansas City Chiefs Zippo Lighter ...,Officially licensed NFL Football Kansas City C...,50000000_Food/Beverage/Tobacco,50210000_Tobacco/Smoking Accessories,50211500_Tobacco Products/Smoking Accessories
128,Emergency Essentials� White Sugar - 90 oz,Whole Wheat Blender Pancakes,50000000_Food/Beverage/Tobacco,50160000_Confectionery/Sugar Sweetening Products,50161500_Sugars/Sugar Substitute Products
131,The Macallan Fine Oak 12 Years 50cl,TASTING NOTES OF The Macallan Fine Oak 12 Year...,50000000_Food/Beverage/Tobacco,50200000_Beverages,50202200_Alcoholic Beverages
161,White Sangria Wine Freezer,Just imagine being in the vineyards of Napa Va...,50000000_Food/Beverage/Tobacco,50200000_Beverages,50202200_Alcoholic Beverages
208,Advantage Milk Chocolate Delight Shake,Product Label 15 g Protein1 g Sugar160 Calorie...,50000000_Food/Beverage/Tobacco,50160000_Confectionery/Sugar Sweetening Products,50161800_Confectionery Products
231,Double-Sided Silver Cigarette Case / Business ...,Small Double-Sided Silver Cigarette Case / Bus...,50000000_Food/Beverage/Tobacco,50210000_Tobacco/Smoking Accessories,50211500_Tobacco Products/Smoking Accessories
283,Kreyol Essence 100% Pure Haitian Black Castor ...,Kreyol Essence 100% Pure Haitian Black Castor ...,50000000_Food/Beverage/Tobacco,50150000_Oils/Fats Edible,50151500_Oils Edible
305,Chocolate Gift Box,A chocoholic's dream! This assortment of cocoa...,50000000_Food/Beverage/Tobacco,50160000_Confectionery/Sugar Sweetening Products,50161800_Confectionery Products
330,Woodstock Mocha Madness Mix1x15lb each,Price as low as $53.45 for bulk buying. Always...,50000000_Food/Beverage/Tobacco,50160000_Confectionery/Sugar Sweetening Products,50161800_Confectionery Products
